In [143]:
import numpy as np
import scipy.integrate as sp
import random as rd
import math as mp
import matplotlib.pyplot as plt
import matplotlib as mpl

In [144]:
steps =  1
birds =  10
H     =    1
beta  =    0.5
ndim  =    3

init_vel_max = 18
init_vel_min = 10
init_radius = 50

x_upper_bound = birds * ndim
v_lower_bound = x_upper_bound
v_upper_bound = 2 * birds * ndim

In [145]:
time_values = np.zeros((steps))
for i in range (steps):
    time_values[i] = i

In [146]:
init_phase_space = np.zeros((birds * 2 * ndim))

In [147]:
def get_pos(bird_set, i, k):
    return bird_set[i + k]
def get_vel(bird_set, i, k):
    return bird_set[birds + i + k]

In [148]:
def pos_diff_norm(bird_set, i, j):
    sum=0
    for k in range(ndim):
        sum = (get_pos(bird_set, i, k) - get_pos(bird_set, j, k))**2
    return sum

In [149]:
def vel_diff_norm(bird_set, i, j):
    sum=0
    for k in range(ndim):
        sum = (get_vel(bird_set, i, k) - get_vel(bird_set, j, k))**2
    return sum

In [150]:
def Aij(bird_set, i, j):
    return H / (pow(1 + pow(vel_diff_norm(bird_set, i, j), 2), beta))

In [151]:
"""
def rhs_equation(t, array):
    vett = np.zeros((birds * 2, ndim))
    for i in range(birds):
        vett[i] = array[i + birds]
    
    j = i
    somma = 0
    while i < (birds * 2):
        while j < (birds * 2):
            if i != j:
                somma += Aij(array[i - birds], array[j - birds]) * (array[j] - array[i])
            j += 1
        vett[i] = somma
        i += 1
    return vett
"""

def rhs_equation(t, phase_space):
    vett = np.zeros(birds * 2 * ndim)    
    
    #equazioni differenziali delle posizioni
    for i in range (birds):
        x_i = i*ndim
        for k in range(ndim):
            vett[x_i + k] = phase_space[i + k]
    
    #equazioni differenziali delle velocità
    for i in range (birds):
        v_i = v_lower_bound + ndim * i
        
        sum = np.zeros((ndim))
        #somma per ogni componente di i
        
        for k in range (ndim):
            for j in range (birds):
                v_jk = get_vel(phase_space, j, k)
                v_ik = get_vel(phase_space, i, k)
                sum[k] += Aij(phase_space, i, j) * (v_jk - v_ik)
            vett[v_i + k] = sum[k]
    return vett

In [153]:
rd.seed()

#inizializzo le posizioni
for i in range (birds):
    x_i = ndim * i
    theta_x = rd.random() * mp.pi
    phi_x = rd.random() * 2 * mp.pi
    r = rd.random() * init_radius
    
    init_phase_space[x_i + 0]= r * mp.cos(phi_x) * mp.sin(theta_x)
    init_phase_space[x_i + 1]= r * mp.sin(phi_x) * mp.sin(theta_x)
    init_phase_space[x_i + 2]= r * mp.cos(theta_x)

#inizializzo le velocità
for i in range (birds):
    v_i = v_lower_bound + ndim * i
    theta_v = rd.random() * mp.pi
    phi_v = rd.random() * 2 * mp.pi
    v = rd.random() * (init_vel_max - init_vel_min) + init_vel_min
    
    init_phase_space[v_i + 0]= v * mp.cos(phi_v) * mp.sin(theta_v)
    init_phase_space[v_i + 1]= v * mp.sin(phi_v) * mp.sin(theta_v)
    init_phase_space[v_i + 2]= v * mp.cos(theta_v)

print (init_phase_space)

[ 20.53688283  -7.68446449  -9.17194521 -11.98328764  44.70872978
   1.08659762  25.23459443  11.55664885 -25.94826497  34.58664008
  25.24534907  -2.48768646   7.93669082  13.27191577  11.05087762
  -1.60039233   8.93586955  -4.61150055   6.22874198  14.44568682
  16.36053958 -34.90263343 -15.54036492  -6.94272295   1.77746712
  -9.94805461 -16.11195701  -0.79222454   6.84798558 -14.25313709
   6.37037964   6.12324582   9.58051356   1.44565392   4.48081927
  14.01971798  15.10936242   1.51620389  -9.64482852  -6.30063728
   3.81229384   9.24477309  -9.7164774    1.9289514   -6.19846918
   2.98421371  -4.6552496  -13.57734697   4.67187792 -10.52976959
   1.69272974   0.3653669   12.75674855  -4.83340606  -2.42525889
   5.95160185  13.06165897   8.5609181   -8.75406368   9.46957342]


In [154]:
#f1 = open(f"vel3D birds={birds} steps={steps} b={beta}.txt", "w")
#f2 = open(f"pos3D birds={birds} steps={steps} b={beta}.txt", "w")
#f1.close()
#f2.close()



solution = sp.solve_ivp(fun=rhs_equation,y0 = init_phase_space, t_span=(0,steps), t_eval = time_values, max_step=0.5)

print(solution.t)
print(solution.y[30:, :])

[0.]
[[  6.37037964]
 [  6.12324582]
 [  9.58051356]
 [  1.44565392]
 [  4.48081927]
 [ 14.01971798]
 [ 15.10936242]
 [  1.51620389]
 [ -9.64482852]
 [ -6.30063728]
 [  3.81229384]
 [  9.24477309]
 [ -9.7164774 ]
 [  1.9289514 ]
 [ -6.19846918]
 [  2.98421371]
 [ -4.6552496 ]
 [-13.57734697]
 [  4.67187792]
 [-10.52976959]
 [  1.69272974]
 [  0.3653669 ]
 [ 12.75674855]
 [ -4.83340606]
 [ -2.42525889]
 [  5.95160185]
 [ 13.06165897]
 [  8.5609181 ]
 [ -8.75406368]
 [  9.46957342]]
